In [1]:
import re
from tqdm import tqdm_notebook
import json

## Перевод из кирилицы в нашу транскрипцию

In [54]:
vowels_semi_pltl = {"ӓ":"a","ӭ":"e","ҍ":""}
vowels_long_pltl = {"я̄":"aa","е̄":"ee","ю̄":"uu","ӣ":"ii"}
vowels_pltl = {"я":"a","е":"e","ю":"u","ё":"o","ь":"","и":"i","ӓ":"a","ӭ":"e","ҍ":""}
vowels_long = {"я̄":"jaa","е̄":"jee","ю̄":"juu","э̄":"ee","ӯ":"uu","а̄":"aa","ӣ":"ii","о̄":"oo"}
vowels = {"ё":"jo","а":"a","ы":"ɨ","у":"u","э":"e","я":"ja","е":"je","ю":"ju","и":"i","о":"o","ӭ":"e","ӓ":"a"}
cnsnt_pltl = {"дз":"ʒ","н":"ɲ","ћ":"h","б":"b","в":"v","г":"g","д":"d","ж":"ž","з":"z","к":"k","л":"l","ӆ":"l̥","м":"m","ӎ":"m̥","ӊ":"n̥","ӈ":"ŋ","п":"p","р":"r","ҏ":"r̥","с":"s","т":"t","ф":"f","х":"x","һ":"h","ш":"š","ц":"c"}
cnsnt_no_pltl = {"дж":"ǯ", "й":"j", "ҋ":"j̥", "ч":"č"}
cnsnts = {"дж":"ǯ","дз":"ʒ","ћ":"h","ҋ":"j̥","й":"j","б":"b","в":"v","г":"g","д":"d","ж":"ž","з":"z","к":"k","л":"l","ӆ":"l̥","м":"m","ӎ":"m̥","н":"n","ӊ":"n̥","ӈ":"ŋ","п":"p","р":"r","ҏ":"r̥","с":"s","т":"t","ф":"f","х":"x","һ":"h","ш":"š","ц":"c","ч":"č","ҍ":"","ь":"","ъ":""}

def get_transcript(word):
    if word == '':
        return ''
    else:
        # заменяем J:
        word = re.sub(r"J", r"j̥", word)
        # делаем все лоуверкейс
        word = word.lower()
        # заменяем полумягкие н:
        for vowel, new_vowel in vowels_semi_pltl.items():
            # сначала долгие, потому что это два символа:
            word = re.sub("нн" + vowel, r"n'n'" + new_vowel, word)
            word = re.sub("н" + vowel, r"n'" + new_vowel, word)
        # заменяем палатальный л:
        word = re.sub(r"льй(\s|$)", r"lij\1", word)
        word = re.sub(r"льй", r"l'j", word)
        # гласные, перед которыми мягкость:
        #     сначала долгие, потому что это два символа на самом деле:
        for vowel, new_vowel in vowels_long_pltl.items():
            for cnsnt, new_cnsnt in cnsnt_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + "'" + new_cnsnt + "'" + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + "'" + new_vowel, word)
            for cnsnt, new_cnsnt in cnsnt_no_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + new_cnsnt + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + new_vowel, word)
        #     короткие гласные, перед которыми мягкость:
        for vowel, new_vowel in vowels_pltl.items():
            for cnsnt, new_cnsnt in cnsnt_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + "'" + new_cnsnt + "'" + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + "'" + new_vowel, word)
            for cnsnt, new_cnsnt in cnsnt_no_pltl.items():
                word = re.sub(cnsnt + cnsnt + vowel, new_cnsnt + new_cnsnt + new_vowel, word)
                word = re.sub(cnsnt + vowel, new_cnsnt + new_vowel, word)
        # заменяем оставшиеся долгие гласные:
        for vowel, new_vowel in vowels_long.items():
            word = re.sub(vowel, new_vowel, word)
        # оставшиеся гласные:
        for vowel, new_vowel in vowels.items():
            word = re.sub(vowel, new_vowel, word)
        # оставшиеся согласные:
        for cnsnt, new_cnsnt in cnsnts.items():
                word = re.sub(cnsnt, new_cnsnt, word)
        return word

## Переводим словарь в json и csv

In [93]:
with open("../sa-ru-kuruch.dsl", encoding="utf-8") as f:
    
    kuruch = f.read()
kuruch = kuruch.splitlines()[3:] # [3:] - обрезаем сверху метаданные

In [4]:
kuruch_list = []
catch_lower = False
# strangers = []
n = -1
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    elif line == '\t[m1][/m]':
        continue
    elif catch_lower:
        catch_lower = False
    elif re.match(r'\t\[m0\]', line):
        continue
    elif re.match(r'\t\[m1\][а-яА-я ]+\[\/m\]', line):
#         strangers.append({'line': line, 'number': n})
        continue
    elif re.match(r'\t\[m1\]([^[ ].+?)(\[b\].*?\[\/b\].*?)?\[/m\]', line) and not re.match('\t\[m1\]([^[ ].+?)\[/b]', line):
        match = re.search(r'\t\[m1\](.*?)(\[b\].*?\[\/b\].*?)?;\[/m\]', line)
        try:
            lost_piece, more_examples = match.groups('')
        except AttributeError:
#             strangers.append({'line': line, 'number': n})
            continue
            
        kuruch_list[-1]['definitions'].append(lost_piece)
        
        if more_examples != '':
            example, definition = re.search(r'\[b\](.*?)\[\/b\](.*?)$', more_examples).groups('')
            example = example.strip()
            item_dict = {'example_cyrillic': example, 
                         'example_transcription': get_transcript(example), 
                         'example_definition': definition.strip()}
            kuruch_list[-1]['examples'].append(item_dict)
    else:
        regex = r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\] ?(?:\(\[b\](?P<form>.*?)\[\/b\]\))? ?\d? ?(?P<common_number>о\. ?ч\. )? ?(?P<word_type>[IV]{1,3})?(?:, (?P<word_alt>\d.))? ?(?P<def>.*?)(?:\[/m\])?$'
        match = re.search(regex, line)
        try:
            word, form, common_number, word_type, word_alt, expanded_definition = match.groups('')
        except AttributeError:
#             strangers.append({'line': line, 'number': n})
            continue
        
        word = re.sub(r'\[.*?\]', r'', word).lower().strip()
        if ' ' in word:
            regex = r'(.*?)(?:\d\.(.*?))?$'
            example_definition, word_definition = re.search(regex, expanded_definition).groups('')
            item_dict = {'example_cyrillic': word, 
                             'example_transcription': get_transcript(word), 
                             'example_definition': example_definition.strip()}
            kuruch_list[-1]['examples'].append(item_dict)
            if word_definition.strip() != '':
                kuruch_list[-1]['definitions'].append(word_definition.strip())
            continue
            
        transcribed_word = get_transcript(word)
        
        if common_number != '':
            common_number = 'общее число'
        
        definitions = []
        examples = []
        for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
            find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
            if find_examples != None:
                example, definition = find_examples.groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())
                    
        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''
        
        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        
        kuruch_list.append(instance)
        n += 1

In [78]:
examples_wrong = '''ама
а̄нсэ
бакалл
бара
воаллтэдтэ
вэстче
вэстчедтэ
вуанчнэ
вӯллькэ
ка̄һтӭ
коатче
кӣрркнэ
куаййвэ
ло̄сстэнне
ме̄ррьдассьтэ
мӣлльтэ
поассэ
пуэдтӭ
роадтэ2
рӯммьп
та̄гкэ
та̄һтэ
цоагчедтэ
че̄д
чоагкэ
чуввнэннтэ
иммель
кӯһт
мӣ
мырр
сӣррэй
то̄лл
ча̄нн
чофта
ке̄
лышэ
манҍтэ
мӣ
мунн
нике̄
туммплэсс
че̄д
чӯдзлэднэ'''.splitlines()

In [79]:
len(examples_wrong)

43

In [84]:
fixing_examples = []
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    if '[ref]' in line:
        continue
    try:
        word, form, common_number, word_type, word_alt, expanded_definition = re.search(r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\] {0,2}(?:\(\[b\](?P<form>.*?)\[\/b\]\))?,? ?\d? ?(?P<common_number>о\. ?ч\. )?[ \t]?(?P<word_type>[IVШ]{1,3})?(?: ?[,.]? *(?P<word_alt>\d ?[^а-яА-я.1\[]))? ?(?P<def>.*?)(?:\[/m\])?$', line).groups('')
    except AttributeError:
        continue
    word = re.sub(r'\[.*?\]', r'', word).lower().strip()
    if word in examples_wrong:
        transcribed_word = get_transcript(word)

        if common_number != '':
            common_number = 'общее число'

        definitions = []
        examples = []
        for item in re.findall(r'(?:\d?\. )?(.*?)(?:[;,]\s?(?=\[b\])|\d\. |$|\\#\s?)', expanded_definition):
            find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
            if find_examples != None:
                example, definition = find_examples.groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())

        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''

        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        fixing_examples.append(instance)

In [85]:
with open('../done/fixed_examples.json', 'w', encoding='utf-8') as json_file:
    json.dump(fixing_examples, json_file, ensure_ascii=False)

In [86]:
with open('../done/fixed_examples.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in fixing_examples:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [4]:
class_wrong = '''адзе
аййдэлсэ
аллькэ
а̄ррвдэллэ
а̄ррьй
а̄ррьп
ба̄нка
билеһтлаһтӭ
бригадирр
бро̄шка
буржуазия
воалнэдтэ
воалхэллэ
вуаннтаслаһтӭ
выррлэ
вя̄ллктэллэ
вя̄ллктэлнэ
килограмм
кӣррькъеннтэ
кывнхэллэ
кышкнэдтэ
кышксэннтэ
лӣб
марксисст
моашшэ
мӯһцтэдтэ
оаҏҏтэдтэ
поаррэ
пӯлле
сайе
таррьв
те̄ҏпчедтэ
ујтнэдтэ
ӯӎтэллэ
цаввцтэдтэ
че̄лнэдтэ
чецкнэдтэ
чоабрнэдтэ
э̄рршассьтэ'''.splitlines()

In [29]:
fixing = []
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    if '[ref]' in line:
        continue
    try:
        word, form, common_number, word_type, word_alt, expanded_definition = re.search(r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\] {0,2}(?:\(\[b\](?P<form>.*?)\[\/b\]\))?,? ?\d? ?(?P<common_number>о\. ?ч\. )?[ \t]?(?P<word_type>[IVШ]{1,3})?(?: ?[,.]? *(?P<word_alt>\d ?[^а-яА-я.1\[]))? ?(?P<def>.*?)(?:\[/m\])?$', line).groups('')
    except AttributeError:
        continue
    word = re.sub(r'\[.*?\]', r'', word).lower().strip()
    if word in class_wrong:
        transcribed_word = get_transcript(word)
        
        if common_number != '':
            common_number = 'общее число'
        
        definitions = []
        examples = []
        for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
            find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
            if find_examples != None:
                example, definition = find_examples.groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())
                    
        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''
        
        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        fixing.append(instance)

In [30]:
with open('../done/fixed_classes.json', 'w', encoding='utf-8') as json_file:
    json.dump(fixing, json_file, ensure_ascii=False)

In [31]:
with open('../done/fixed_classes.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in fixing:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [94]:
stems_wrong = '''алльк
алфавиһт
апрелль
а̄ррв
а̄ррьв
а̄һклэӊӊк
батонн
билеһт
бӣннт
воаррлэнч
е̄ммьне
оаз
оаз
па̄ццк
пӣжь
рӯсст
рӯһц
рӯһц
са̄ррьк
са̄ррьк
суннҍт
суннҍт
та̄йймушш
та̄лльв
тёллэдтэ
тӣрьмесь
то̄ленч
тӯвас
тӯйй
це̄гкь
ча̄дзь
че̄гар
э̄д
ялл
тӯййлаһтӭ'''.splitlines()

In [95]:
len(stems_wrong)

35

In [96]:
fixing_stems = []
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    if '[ref]' in line:
        continue
    try:
        word, form, common_number, word_type, word_alt, expanded_definition = re.search(r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\][ \t\d]{0,3}(?:\((?:\[b\])?(?P<form>.*?)(?:\[\/b\])?\))?,? ?\d? ?(?P<common_number>о\. ?ч\. )?[ \t]?(?P<word_type>[IVШ]{1,3})?(?: ?[,.]? *(?P<word_alt>\d ?[^а-яА-я.1\[]))? ?(?P<def>.*?)(?:\[/m\])?$', line).groups('')
    except AttributeError:
        continue
    word = re.sub(r'\[.*?\]', r'', word).lower().strip()
    if word in stems_wrong:
        transcribed_word = get_transcript(word)
        
        if common_number != '':
            common_number = 'общее число'
        
        definitions = []
        examples = []
        for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
            find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
            if find_examples != None:
                example, definition = find_examples.groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())
                    
        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''
        
        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        fixing_stems.append(instance)

In [97]:
with open('../done/fixed_stems.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in fixing_stems:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [98]:
with open('../done/fixed_stems.json', 'w', encoding='utf-8') as json_file:
    json.dump(fixing_stems, json_file, ensure_ascii=False)

In [110]:
line = '[m1][b]ця̄һкэ [/b]I, 3 1. проводить / провести черту, линию 2. разрубать / разрубить что-л. на куски; [b]ця̄к вуэнч ке̄ммна[/b]! разруби мясо на куски для котла! 3. метить / пометить (оленей, вырезать ножом на боку оленя родовой знак); [b]ця̄һкэ пӯдзэтҍ[/b]! пометьте оленей[/m]'
try:
    word, form, common_number, word_type, word_alt, expanded_definition = re.search(r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\][ \t\d]{0,3}(?:\((?:\[b\])?(?P<form>.*?)(?:\[\/b\])?\))?,? ?\d? ?(?P<common_number>о\. ?ч\. )?[ \t]?(?P<word_type>[IVШ]{1,3})?(?: ?[,.]? *(?P<word_alt>\d ?[^а-яА-я.1\[]))? ?(?P<def>.*?)(?:\[/m\])?$', line).groups('')
except AttributeError:
    pass

word = re.sub(r'\[.*?\]', r'', word).lower().strip()
transcribed_word = get_transcript(word)

if common_number != '':
    common_number = 'общее число'

definitions = []
examples = []
for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
    find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
    if find_examples != None:
        example, definition = find_examples.groups('')
        example = example.strip()
        item_dict = {'example_cyrillic': example, 
                     'example_transcription': get_transcript(example), 
                     'example_definition': definition.strip()}
        examples.append(item_dict)
    else:
        if item.strip() != '':
            definitions.append(item.strip())

if word_type != '':
    if transcribed_word.endswith('e'):
        pos = 'Verb'
    else:
        pos = 'Noun'
else:
    pos = ''

instance = {'cyrillic_backwards': word[::-1],
            'transcription_backwards': transcribed_word[::-1],
            'cyrillic': word,
            'transcription': transcribed_word,
            'derived_cyrillic': main_word,
            'derived_transcription': transcribed_main_word,
            'form_cyrillic': form,
            'form_transcription': get_transcript(form),
            'pos': pos,
            'common_number': common_number,
            'type': word_type,
            'type_alternation': word_alt,
            'definitions': definitions,
            'examples' : examples
            }
examples = ''
for example in instance['examples']:
    examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
print(f"{instance['cyrillic_backwards']}\t{instance['transcription_backwards']}\t{instance['cyrillic']}\t{instance['transcription']}\t{instance['derived_cyrillic']}\t{instance['derived_transcription']}\t{instance['form_cyrillic']}\t{instance['form_transcription']}\t{instance['pos']}\t{instance['common_number']}\t{instance['type']}\t{instance['type_alternation']}\t{';'.join(instance['definitions'])}\t{examples}\n")

экһ̄яц	ekhaa'c	ця̄һкэ	c'aahke	я̄рр	jaarr			Verb		I	3 	проводить / провести черту, линию;разрубать / разрубить что-л. на куски;метить / пометить (оленей, вырезать ножом на боку оленя родовой знак)	ця̄к вуэнч ке̄ммна	c'aak vuenč k'eemmna	! разруби мясо на куски для котла!	ця̄һкэ пӯдзэтҍ	c'aahke puuʒet	! пометьте оленей	



In [56]:
fixing_lost = []
for line in tqdm_notebook(kuruch):
    if not re.match(r'\t', line):
        # в начале статьи нет табулиции
        if line.islower():
            # дериваты, с ними работаем отдельно в каждой статье, нет смысла отдельно разбирать
            catch_lower = True
            continue
        main_word = re.sub(r'\d',r'', line.lower()).strip()
        transcribed_main_word = get_transcript(main_word)
    if '[ref]' in line:
        continue
    if re.match('\t\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\](?P<lost>[а-ю])\W ?(?:\(\[b\](?P<form>.*?)\[\/b\]\))? ?\d? ?(?P<common_number>о\. ?ч\. )? ?(?P<word_type>[0-9IVШ]{1,3})?(?:,? ?(?P<word_alt>\d ?\W))? ?(?P<def>.*?)(?:\[/m\])?$', line):
        word, lost, form, common_number, word_type, word_alt, expanded_definition = re.search(r'\[m1\] ?(?:\[b\])?(?P<word>.*?)\[\/b\](?P<lost>[а-ю])\W ?(?:\(\[b\](?P<form>.*?)\[\/b\]\))? ?\d? ?(?P<common_number>о\. ?ч\. )? ?(?P<word_type>[0-9IVШ]{1,3})?(?:,? ?(?P<word_alt>\d ?\W))? ?(?P<def>.*?)(?:\[/m\])?$', line).groups('')
        word = re.sub(r'\[.*?\]', r'', word).lower().strip() + lost
        if ' ' in word:
            continue
        transcribed_word = get_transcript(word)

        if common_number != '':
            common_number = 'общее число'

        definitions = []
        examples = []
        for item in re.findall(r'(?:\d\. )?(.*?)(?:;\s?|\d\. |$|\\#\s?)', expanded_definition):
            find_examples = re.search(r'^\[b\](?P<example>.*?)\[\/b\](?P<example_def>.*?)$', item)
            if find_examples != None:
                example, definition = find_examples.groups('')
                example = example.strip()
                item_dict = {'example_cyrillic': example, 
                             'example_transcription': get_transcript(example), 
                             'example_definition': definition.strip()}
                examples.append(item_dict)
            else:
                if item.strip() != '':
                    definitions.append(item.strip())

        if word_type != '':
            if transcribed_word.endswith('e'):
                pos = 'Verb'
            else:
                pos = 'Noun'
        else:
            pos = ''

        instance = {'cyrillic_backwards': word[::-1],
                    'transcription_backwards': transcribed_word[::-1],
                    'cyrillic': word,
                    'transcription': transcribed_word,
                    'derived_cyrillic': main_word,
                    'derived_transcription': transcribed_main_word,
                    'form_cyrillic': form,
                    'form_transcription': get_transcript(form),
                    'pos': pos,
                    'common_number': common_number,
                    'type': word_type,
                    'type_alternation': word_alt,
                    'definitions': definitions,
                    'examples' : examples
                    }
        fixing_lost.append(instance)

In [57]:
len(fixing_lost)

2

In [59]:
with open('../done/fixed_lost.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in fixing_lost:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [60]:
with open('../done/fixed_lost.json', 'w', encoding='utf-8') as json_file:
    json.dump(fixing_lost, json_file, ensure_ascii=False)

In [5]:
# первый раз

# for stranger in strangers:
#     print(stranger, end='\n\n')
#     print(kuruch_list[stranger['number']]['definitions'])
#     print(kuruch_list[stranger['number']]['examples'])
#     decision = str(input())
#     stranger['decision'] = decision
# with open('../intermediate files/strangers.json', 'w', encoding='utf-8') as json_file:
#     json.dump(strangers, json_file, ensure_ascii=False)

# не первый раз
with open('../intremediate files/strangers.json', encoding='utf-8') as f:
    strangers = json.load(f)

In [6]:
for stranger in strangers:
    n = stranger['number']
    if 'decision' not in stranger:
        print(stranger['line'])
        print(kuruch_list[n])
        stranger['decision'] = str(input())
    if stranger['decision'] == 'part of last definition':
        part = re.search(r'\t\[m1\](.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['definitions'][-1] += ' ' + part
    elif stranger['decision'] == 'part of last definition and new definition':
        lost_part, new_def = re.search(r'\t\[m1\](.*?);? ?\d\. (.*?)\[\/m\]', stranger['line']).groups('')
        kuruch_list[n]['definitions'][-1] += ' ' + lost_part
        kuruch_list[n]['definitions'].append(new_def.strip())
    elif stranger['decision'] == 'part of last example definition':
        part = re.search(r'\t\[m1\](.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['examples'][-1]['example_definition'] += ' ' + part
    elif stranger['decision'] == 'new definition for last item':
        part = re.search(r'\t\[m1\] ?\d\. (.*?)\[\/m\]', stranger['line']).group()
        kuruch_list[n]['definitions'].append(part)
    elif stranger['decision'] == 'new definitions for last item':
        parts = re.findall(r'\d\. (.*?)(?:\[\/m\]|\d)', stranger['line'])
        for part in parts:
            kuruch_list[n]['definitions'].append(part.strip())
    elif stranger['decision'] == 'part of last definition and new definitions':
        lost_part, new = re.search(r'\t\[m1\](.*?)(\d\..*?)\[\/m\]', stranger['line']).groups('')
        kuruch_list[n]['definitions'][-1] += ' ' + lost_part.strip()
        parts = re.findall(r'\d\. (.*?)(?:$|\d)', new)
        for part in parts:
            kuruch_list[n]['definitions'].append(part.strip())
    elif stranger['decision'] == 'part of last definition and new definitions and new example':
        print(stranger['line'])
        kuruch_list[n]['definitions'][-1] += ' ' + str(input('lost part: '))
        example = str(input('example: '))
        kuruch_list[n]['examples'].append({'example_cyrillic': example, 
                                             'example_transcription': get_transcript(example), 
                                             'example_definition': str(input('definition: '))})
    elif stranger['decision'] == 'part of last example and new definition':
        print(stranger['line'])
        kuruch_list[n]['examples'][-1]['example_definition'] += ' ' + str(input('lost part: '))
        new = str(input('new definition: '))
        kuruch_list[n]['definitions'].append(new)

	[m1]иногда, бывало); пудзэ чёлнэв чурьвэть чёххча олени обдирают рога осенью[/m]
lost part: иногда, бывало)
example: пудзэ чёлнэв чурьвэть чёххча
definition: олени обдирают рога осенью
	[m1]затишье 2. перен. успокаиваться / успокоиться (после шума, скандала)[/m]
lost part: затишье
new definition: перен. успокаиваться / успокоиться (после шума, скандала)
	[m1]в тундру[/m]
{'cyrillic_backwards': 'ейыв', 'transcription_backwards': 'ejɨv', 'cyrillic': 'выйе', 'transcription': 'vɨje', 'derived_cyrillic': 'выйе', 'derived_transcription': 'vɨje', 'form_cyrillic': '', 'form_transcription': '', 'pos': 'Verb', 'common_number': '', 'type': 'I', 'type_alternation': '', 'definitions': ['ехать'], 'examples': [{'example_cyrillic': 'выйе пӯдзэ альн', 'example_transcription': "vɨje puuʒe al'n", 'example_definition': 'ехать на оленях'}, {'example_cyrillic': 'выйе по̄йсэсьт', 'example_transcription': "vɨje poojses't", 'example_definition': 'ехать поездом'}, {'example_cyrillic': 'выйем ве̄шш', 'example_t

In [7]:
kuruch_list[8]

{'cyrillic_backwards': 'эрбба',
 'transcription_backwards': 'erbba',
 'cyrillic': 'аббрэ',
 'transcription': 'abbre',
 'derived_cyrillic': 'аббьр',
 'derived_transcription': "ab'b'r",
 'form_cyrillic': '',
 'form_transcription': '',
 'pos': 'Verb',
 'common_number': '',
 'type': 'II',
 'type_alternation': '4 ',
 'definitions': ['идти - о дожде, дождить'],
 'examples': [{'example_cyrillic': 'ке̄сся тоаййв эбьрэ',
   'example_transcription': "k'ees's'a toajjv eb're",
   'example_definition': 'лётом часто шёл дождь'}]}

In [8]:
with open('../done/kuruch.csv','w',encoding='utf-8') as f:
    f.write('cyrillic_backwards\ttranscription_backwards\tcyrillic\ttranscription\tderived_cyrillic\tderived_transcription\tform_cyrillic\tform_transcription\tpos\tcommon_number\ttype\ttype_alternation\tdefinitions\texample_cyrillic\texample_transcription\texample_definition\n')
    for word in kuruch_list:
        examples = ''
        for example in word['examples']:
            examples += f"{example['example_cyrillic']}\t{example['example_transcription']}\t{example['example_definition']}\t"
        line = f"{word['cyrillic_backwards']}\t{word['transcription_backwards']}\t{word['cyrillic']}\t{word['transcription']}\t{word['derived_cyrillic']}\t{word['derived_transcription']}\t{word['form_cyrillic']}\t{word['form_transcription']}\t{word['pos']}\t{word['common_number']}\t{word['type']}\t{word['type_alternation']}\t{';'.join(word['definitions'])}\t{examples}\n"
        f.write(line)

In [9]:
with open('../done/kuruch.json', 'w', encoding='utf-8') as json_file:
    json.dump(kuruch_list, json_file, ensure_ascii=False)

## Мостерская ноябрь 2020. Исправление ошибок.

In [3]:
import pandas as pd

In [310]:
kuruch = pd.read_csv('../done/kuruch_revised_november.csv', encoding='utf-8')

In [311]:
kuruch.drop_duplicates(inplace=True)

In [312]:
kuruch.head(2)

,cyrillic_backwards,transcription_backwards,cyrillic,transcription,derived_cyrillic,derived_transcription,form_cyrillic,form_transcription,часть речи,grammar,...,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,львв̄а,l'v'vaa,а̄ввьл,aav'v'l,а̄ввьл,aav'v'l,а̄вьл,aav'l,Noun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,лл̄ат,llaat,та̄лл,taall,та̄лл,taall,та̄л,taal,Noun,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
kuruch.columns[:16]

Index(['cyrillic_backwards', 'transcription_backwards', 'cyrillic',
       'transcription', 'derived_cyrillic', 'derived_transcription',
       'form_cyrillic', 'form_transcription', 'часть речи', 'grammar', 'type',
       'type_alternation', 'definition', 'example_cyrillic',
       'example_transcription', 'example_definition'],
      dtype='object')

### Транскрипция

In [314]:
# ɲ' -> ɲ

for column in ['transcription','derived_transcription','form_transcription']:
    kuruch[column] = kuruch[column].apply(lambda x: x.replace("ɲ'","ɲ") if isinstance(x, str) else x)

kuruch['transcription_backwards'] = kuruch['transcription_backwards'].apply(lambda x: x.replace("'ɲ","ɲ") if isinstance(x, str) else x)

In [315]:
# ӭäҍ -> палатализация

def semi_palatal(row):
    for column in ['','derived_','form_']:
        cyr = row[f'{column}cyrillic']
        if (isinstance(cyr, str)) and (('ä' in cyr) or ('ӭ' in cyr) or ('ҍ' in cyr)):
            row[f'{column}transcription'] = get_transcript(cyr)
    return row

In [316]:
kuruch = kuruch.apply(semi_palatal, axis=1)

In [317]:
# dt -> dd
# dt' -> d'd'
# bp -> bb
# bp' -> b'b'
# gk -> gg
# gk' -> g'g'

def repl(g):
    return g.group(1)+g.group(2)+g.group(1)+g.group(2)

for column in ['','derived_','form_']:
    kuruch[f'{column}transcription'] = kuruch[f'{column}transcription'].str.replace(r"([dbg])[tpk]('?)", repl)

In [318]:
kuruch['transcription_backwards'] = kuruch['transcription'].apply(lambda x: x[::-1])

### Тем словам, у которых определение начинается на "то же, что Х" скопировать определения, часть речи и grammar

In [319]:
def repeat_link(row):
    if 'то же, что' in row.definition:
        link = re.search(r'то же, что(?P<link>.*?)$', row.definition)
        if (link != None):
            link = link.group('link').replace('[b]','')
            link = re.sub(r'\d',r'', link).strip()
            if link in kuruch.cyrillic.values:
                v = kuruch[kuruch.cyrillic == link][['definition','часть речи','grammar']].values
                if v.shape[0] > 1:
                    print(link, 'appears more than 1 time')
                    for definition in v:
                        print('\t',definition[0])
                for i,value in enumerate(v[0]):
                    row[['definition','часть речи','grammar'][i]] = value
            else:
                print(link, 'is not in kuruch')
        else:
            print(row.cyrillic, row.definition)
    return row

In [320]:
kuruch[kuruch.cyrillic == 'ко̄днэ'][['definition','часть речи','grammar']]

,definition,часть речи,grammar
1682,"то же, что [b]ко̄дче",Verb,NaN
1683,"нереститься (постоянно;иногда, бывало)",Verb,"постоянно;иногда, бывало"


In [321]:
kuruch = kuruch.apply(repeat_link, axis=1)

адтҍенч is not in kuruch
амшуввлэ is not in kuruch
бинтнэ is not in kuruch
ва̄йймлассьтэ is not in kuruch
ва̄ллтьюввэ is not in kuruch
ёадтэдтӭ is not in kuruch
е̄ххьтлэнч е̄ххьтлэнч is not in kuruch
ке̄ллъе is not in kuruch
ко̄зсэ appears more than 1 time
	 поуправлять кем-чем (немного)
	 скосить что (быстро)
ко̄днэ appears more than 1 time
	 вязать (постоянно, иногда, бывало)
	 нереститься (постоянно;иногда, бывало)
ко̄ммтнэ appears more than 1 time
	 то же, что [b]ко̄ммтнэ
	 стать более широким, расшириться;разноситься (об обуви);разлиться (о реках)
ко̄зсэ appears more than 1 time
	 поуправлять кем-чем (немного)
	 скосить что (быстро)
кудас кудтэмплоагкь is not in kuruch
кэ̄ррь-каћтӭ is not in kuruch
ла̄шькэсь is not in kuruch
лоаjтнэ is not in kuruch
ло̄ћктуввэ is not in kuruch
ме̄ррьктэллэ is not in kuruch
наввьлдэ is not in kuruch
ояффклэ is not in kuruch
оафкнэ is not in kuruch
па̄гкье is not in kuruch
пэ̄днэ is not in kuruch
ростхуввэ is not in kuruch
сыйпнэ is not in kuruch
сэ

In [322]:
kuruch.sample(10).definition

533                      бывать (где-л. - иногда, бывало)
7069    ловить / поймать тж. перен., вылавливать / выл...
5814    считать что-л. очень полным;считать, что где-л...
80                           успокаивать кого (постоянно)
2845                                               поздно
1523    разматываться (о намотанном - иногда, бывало);...
5613                                               ошибка
70                      теперь, сейчас, в настоящее время
5175                                              темнота
3048                       кислить, иметь кисловатый вкус
Name: definition, dtype: object

In [323]:
kuruch.definition[kuruch.definition.str.contains('\[')]

43                                   страд. к [b]а̄ввьлье
117                                   то же, что [b]а̄ллэ
124     задний отросток \[на середине\] рога (оленя);о...
139                                     понуд. к [b]альнэ
150                                    однокр. [b]зевнуть
                              ...                        
7333                                   понуд. к [b]юссьтэ
7338                                     страд. к [b]юһкэ
7349                                    страд. к [b]ю̄гке
7367                                  страд. к [b]я̄ккъсэ
7390                                         \[моя\] мама
Name: definition, Length: 542, dtype: object

In [324]:
kuruch.replace(r'\[\/?b\]',r'', regex=True,inplace=True)
kuruch.replace(r'\\\[',r'', regex=True,inplace=True)
kuruch.replace(r'\\\]',r'', regex=True,inplace=True)

In [325]:
kuruch.to_csv('../done/kuruch_revised_november_fixed.csv', encoding='utf-8', index=False)

In [326]:
kuruch.definition[kuruch.definition.str.contains('\[')]

735                           понуд. от [ref]ВУЭДТӬ[/ref]
1742            ход (четыре койбы, см. [ref]КОАММАС[/ref]
1807    крюк для выделки шкур, койб (см. [ref]КОАММАС[...
2414    липты (длинные меховые чулки шерстью внутрь, п...
2590                  место в веже (см. [ref]КУЭДТҌ[/ref]
Name: definition, dtype: object